Install Requirements ()

In [1]:
!pip install autogen-agentchat autogen-ext[openai] -q
!pip install groq -q
!pip install arxiv -q
!pip install PyPDF2 -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.9/83.9 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 234.0/234.0 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.1/65.1 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.7/319.7 kB 15.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opentelemetry-sdk 1.16.0 requires opentelemetry-api==1.16.0, but you have opentelemetry-api 1.31.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.2/122.2 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat, SelectorGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_core.tools import FunctionTool
from autogen_agentchat.conditions import MaxMessageTermination
import arxiv

In [3]:
import os
from getpass import getpass

tokenGROQ = getpass('Enter GROQ_API_KEY here: ')
os.environ["GROQ_API_KEY"] = tokenGROQ

Enter GROQ_API_KEY here: ··········


In [6]:
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient

# Create the agents.
custom_model_client = OpenAIChatCompletionClient(
    model="llama-3.3-70b-versatile",
    base_url="https://api.groq.com/openai/v1",
    api_key=os.environ["GROQ_API_KEY"],
    model_info={
        "vision": False,
        "function_calling": True,
        "json_output": False,
        "family": "unknown",
    },
)
assistant = AssistantAgent("assistant", model_client=custom_model_client)
user_proxy = UserProxyAgent("user_proxy", input_func=input)  # Use input() to get user input from console.

# Create the termination condition which will end the conversation when the user says "APPROVE".
termination = TextMentionTermination("DONE")

# Create the team.
team = RoundRobinGroupChat([assistant, user_proxy], termination_condition=termination)

# Run the conversation and stream to the console.
stream = team.run_stream(task="the pdf file is a little bit large, can you read it if i sent it to you completely, or you need it in small chunks?")
await Console(stream)

---------- user ----------
the pdf file is a little bit large, can you read it if i sent it to you completely, or you need it in small chunks?
---------- assistant ----------
I'm a large language model, I don't have the capability to directly receive or process files, including PDFs. I can only process text-based input. If you'd like to share the content of the PDF, you can copy and paste the relevant text into this chat window, and I'll do my best to assist you.

If the PDF is too large to copy and paste, you can also consider breaking it down into smaller sections or summarizing the main points, and I can help you with that.

Please let me know how I can assist you further.
Enter your response: DONE
---------- user_proxy ----------
DONE


TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='the pdf file is a little bit large, can you read it if i sent it to you completely, or you need it in small chunks?', type='TextMessage'), TextMessage(source='assistant', models_usage=RequestUsage(prompt_tokens=88, completion_tokens=115), metadata={}, content="I'm a large language model, I don't have the capability to directly receive or process files, including PDFs. I can only process text-based input. If you'd like to share the content of the PDF, you can copy and paste the relevant text into this chat window, and I'll do my best to assist you.\n\nIf the PDF is too large to copy and paste, you can also consider breaking it down into smaller sections or summarizing the main points, and I can help you with that.\n\nPlease let me know how I can assist you further.", type='TextMessage'), UserInputRequestedEvent(source='user_proxy', models_usage=None, metadata={}, request_id='eb38c076-59fb-4908-9821-

In [10]:
import arxiv

# Construct the default API client.
client = arxiv.Client()
papers_ids = []

# Search for the 10 most recent articles matching the keyword "quantum."
search = arxiv.Search(
  query = "Multi-agent LLM systems",
  max_results = 1,
  sort_by = arxiv.SortCriterion.SubmittedDate
)

for result in client.results(search):
  result.download_pdf(filename=f"{result.get_short_id()}.pdf")
  papers_ids.append(result.get_short_id())
  print(f"\nTitle: {result.get_short_id()}")


Title: 2503.10630v1


In [11]:
paper = next(arxiv.Client().results(arxiv.Search(id_list=["2503.10630v1"])))
print(paper.get_short_id())

2503.10630v1


In [12]:
import PyPDF2
import openai

# Step 1: Extract text from the PDF
pdf_path = "/content/2503.10630v1.pdf"  # Adjust the path as needed
paper_text = ""
with open(pdf_path, "rb") as f:
    reader = PyPDF2.PdfReader(f)
    for page in reader.pages:
        page_text = page.extract_text()
        if page_text:
            paper_text += page_text + "\n"

# Step 2: Split the text into manageable chunks
def chunk_text(text, max_length=2500):
    """Splits text into chunks of approximately max_length characters."""
    return [text[i:i+max_length] for i in range(0, len(text), max_length)]


chunks = chunk_text(paper_text, max_length=2500)


In [13]:
print(len(chunks))

24


In [14]:
def word_count(texts):
  total_words = 0
  total_characters = 0

  for doc in texts:
      content = doc
      word_count = len(content.split())  # Count words
      char_count = len(content)         # Count characters
      total_words += word_count
      total_characters += char_count

  num_docs = len(texts)
  avg_words = total_words / num_docs if num_docs > 0 else 0
  avg_characters = total_characters / num_docs if num_docs > 0 else 0

  print(f"Average words per document: {avg_words}")
  print(f"Average characters per document: {avg_characters}")

word_count(chunks)

Average words per document: 381.2916666666667
Average characters per document: 2400.0833333333335


In [15]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.schema import Document

# Convert text into Document objects
documents = [Document(page_content=text) for text in chunks]

# Initialize text splitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

# Split the documents correctly
splits = text_splitter.split_documents(documents)

In [16]:
!pip install langchain-community rank_bm25 -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.9 MB/s eta 0:00:00


In [18]:
from langchain.retrievers import BM25Retriever

bm25_retriever_split = BM25Retriever.from_documents(splits)

In [19]:
query = "What are the components of agent systems?"

results = bm25_retriever_split.invoke(query)

print("BM25 Retrieved Results:")
for i, doc in enumerate(results):
    print(f"Result {i+1}:\n{doc.page_content}\n")

BM25 Retrieved Results:
Result 1:
pagation 𝑡- th Propagation(a)
(b)
LLM
Scene Graph CorrectionFigure 3. Illustration of approach. (a) Stage 2: coordinate projec-
tion and anchor pair alignment. (b) Stage 3: scene graph correction.
gradually infer the projected BEV coordinates of the whole
graph. During the traversal, we focus on the edge connecting
last node (coordinate-known) and current node (coordinate-
unknown), which stores the spatial relationship. For each
edge, we prompt LLM with: [Current Node] is [Relation-
ship] [Last Node], coordinate of [Last Node] is (x, y), the
X-axis and Y-axis are positive towards yourself and the right
respectively. What is the coordinate of [Current Node]? to
project the current coordinate-unknown node to BEV .
With the projected GtandGg, we conduct alignment
for each anchor pair in order. Assume that the anchors
arev1
t,v2
t∈ Gtandv1
g,v2
g∈ Gg.P∈R3×3is the 2D
coordinate transfer matrix, consisting of scale S, rotation R
and translation T, which is 

In [4]:

!pip install pymupdf -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 73.9 MB/s eta 0:00:00


In [25]:
import fitz  # PyMuPDF

def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text() + "\n"
    return text

pdf_text = extract_text_from_pdf("/content/2503.10630v1.pdf")

In [20]:
import nltk
from nltk.tokenize import sent_tokenize

nltk.download("punkt")
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [26]:
def chunk_text(text, chunk_size=500):
    sentences = sent_tokenize(text)
    chunks = []
    current_chunk = ""

    for sentence in sentences:
        if len(current_chunk) + len(sentence) < chunk_size:
            current_chunk += " " + sentence
        else:
            chunks.append(current_chunk)
            current_chunk = sentence
    chunks.append(current_chunk)  # Last chunk
    return chunks

chunks = chunk_text(pdf_text, chunk_size=1024)

In [27]:
print(len(chunks))

62


In [28]:
def arxiv_search(chunk_index: int) -> list:  # type: ignore[type-arg]
    return chunks[chunk_index * 22:(chunk_index + 1) * 22]

arxiv_search_tool = FunctionTool(
    arxiv_search, description="get the paper content"
)

arxiv_search_agentt = AssistantAgent(
    name="Arxiv_Search_Agent",
    tools=[arxiv_search_tool],
    model_client=OpenAIChatCompletionClient(
        model="llama-3.3-70b-versatile",
        base_url="https://api.groq.com/openai/v1",
        api_key=os.environ["GROQ_API_KEY"],
        model_info={
            "vision": False,
            "function_calling": True,
            "json_output": False,
            "family": "unknown",
        },
    ),
    description="An agent that can get the paper content",
    system_message="You are a helpful AI assistant. Solve tasks using your tools..",
)

summarizeAgent = AssistantAgent(
    name="summarize_Agent",
    model_client=OpenAIChatCompletionClient(
        model="llama-3.3-70b-versatile",
        base_url="https://api.groq.com/openai/v1",
        api_key=os.environ["GROQ_API_KEY"],
        model_info={
            "vision": False,
            "function_calling": True,
            "json_output": False,
            "family": "unknown",
        },
    ),
    description="An agent that can summarize papers.",
    system_message="You are a helpful AI assistant. A paper given to you please summarize it SHORTLY AND PRECISELY.",
)

text_mention_termination = TextMentionTermination("TERMINATE")
max_messages_termination = MaxMessageTermination(max_messages=5)
termination = text_mention_termination | max_messages_termination

team = RoundRobinGroupChat(
    participants=[arxiv_search_agentt, summarizeAgent], termination_condition=termination
)

async def summarize_large_paper():
    chunk_index = 0
    final_summary = ""
    while True:
        # Fetch the next chunk
        chunk = arxiv_search(chunk_index)
        if not chunk:
            break  # Exit the loop if there are no more chunks

        # Summarize the current chunk
        summary = await team.run_stream(
            task=f"Summarize this chunk of the paper: {chunk}"
        )
        final_summary += summary + "\n"
        chunk_index += 1

    # Final summarization of all chunk summaries
    final_summary = await team.run_stream(
        task=f"Summarize the following summaries into one concise summary: {final_summary}"
    )
    return final_summary

# Run the summarization process
final_summary = await summarize_large_paper()
print(final_summary)

TypeError: object async_generator can't be used in 'await' expression

In [31]:
def arxiv_search() -> list:  # type: ignore[type-arg]
    return chunks[:22]

arxiv_search_tool = FunctionTool(
    arxiv_search, description="get the paper content"
)

arxiv_search_agentt = AssistantAgent(
    name="Arxiv_Search_Agent",
    tools=[arxiv_search_tool],
    model_client = OpenAIChatCompletionClient(
      model="llama-3.3-70b-versatile",
      base_url="https://api.groq.com/openai/v1",
      api_key=os.environ["GROQ_API_KEY"],
      model_info={
        "vision": False,
        "function_calling": True,
        "json_output": False,
        "family": "unknown",
      },
    ),
    description="An agent that can get the paper content",
    system_message="You are a helpful AI assistant. Solve tasks using your tools..",
)


summarizeAgent = AssistantAgent(
    name="summarize_Agent",
    model_client = OpenAIChatCompletionClient(
      model="llama-3.3-70b-versatile",
      base_url="https://api.groq.com/openai/v1",
      api_key=os.environ["GROQ_API_KEY"],
      model_info={
        "vision": False,
        "function_calling": True,
        "json_output": False,
        "family": "unknown",
      },
    ),
    description="An agent that can summarize papers.",
    system_message="You are a helpful AI assistant. a paper given to you please summarize it SHORTLY AND PRECISELY.",
)

text_mention_termination = TextMentionTermination("TERMINATE")
max_messages_termination = MaxMessageTermination(max_messages=5)
termination = text_mention_termination | max_messages_termination


team = RoundRobinGroupChat(
    participants=[arxiv_search_agentt, summarizeAgent], termination_condition=termination
)

await Console(
    team.run_stream(
        task="give me the summarization of the given paper",
    )
)

---------- user ----------
give me the summarization of the given paper
---------- Arxiv_Search_Agent ----------
[FunctionCall(id='call_rzjq', arguments='{}', name='arxiv_search')]
---------- Arxiv_Search_Agent ----------
[FunctionExecutionResult(content="[' UniGoal: Towards Universal Zero-shot Goal-oriented Navigation\\nHang Yin1*, Xiuwei Xu1∗†, Linqing Zhao1, Ziwei Wang2, Jie Zhou1, Jiwen Lu1‡\\n1Tsinghua University\\n2Nanyang Technological University\\n{yinh23, xxw21, zhaolinqing}@mails.tsinghua.edu.cn;\\nziwei.wang@ntu.edu.sg; {jzhou, lujiwen}@tsinghua.edu.cn\\nAbstract\\nIn this paper, we propose a general framework for univer-\\nsal zero-shot goal-oriented navigation. Existing zero-shot\\nmethods build inference framework upon large language\\nmodels (LLM) for specific tasks, which differs a lot in over-\\nall pipeline and fails to generalize across different types of\\ngoal. Towards the aim of universal zero-shot navigation, we\\npropose a uniform graph representation to unify d

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='give me the summarization of the given paper', type='TextMessage'), ToolCallRequestEvent(source='Arxiv_Search_Agent', models_usage=RequestUsage(prompt_tokens=231, completion_tokens=10), metadata={}, content=[FunctionCall(id='call_rzjq', arguments='{}', name='arxiv_search')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(source='Arxiv_Search_Agent', models_usage=None, metadata={}, content=[FunctionExecutionResult(content="[' UniGoal: Towards Universal Zero-shot Goal-oriented Navigation\\nHang Yin1*, Xiuwei Xu1∗†, Linqing Zhao1, Ziwei Wang2, Jie Zhou1, Jiwen Lu1‡\\n1Tsinghua University\\n2Nanyang Technological University\\n{yinh23, xxw21, zhaolinqing}@mails.tsinghua.edu.cn;\\nziwei.wang@ntu.edu.sg; {jzhou, lujiwen}@tsinghua.edu.cn\\nAbstract\\nIn this paper, we propose a general framework for univer-\\nsal zero-shot goal-oriented navigation. Existing zero-shot\\nmethods build inference framewo

**1. Research Agent (Paper Retriever)**

    Purpose: Searches for and downloads relevant academic papers from databases (e.g., Google Scholar, ArXiv, Semantic Scholar).
    Responsibilities:
    1.   Querying research databases with relevant keywords.
    2.   Downloading full-text articles or abstracts.
    3.   Storing references in a structured manner (e.g., BibTeX or JSON format).

In [ ]:
def arxiv_search(query: str, max_results: int = 2) -> list:  # type: ignore[type-arg]
    """
    Search Arxiv for papers and return the results including abstracts.
    """
    import arxiv

    client = arxiv.Client()
    search = arxiv.Search(query=query, max_results=max_results, sort_by=arxiv.SortCriterion.Relevance)
    print("************ calling")
    results = []
    for paper in client.results(search):
        results.append(
            {
                "title": paper.title,
            }
        )

    return results

In [ ]:

arxiv_search_tool = FunctionTool(
    arxiv_search, description="Search Arxiv for papers related to a given topic, including abstracts"
)

In [ ]:
arxiv_search_agent = AssistantAgent(
    name="Arxiv_Search_Agent",
    tools=[arxiv_search_tool],
    model_client = OpenAIChatCompletionClient(
      model="llama-3.3-70b-versatile",
      base_url="https://api.groq.com/openai/v1",
      api_key=os.environ["GROQ_API_KEY"],
      model_info={
        "vision": False,
        "function_calling": True,
        "json_output": False,
        "family": "unknown",
      },
    ),
    description="An agent that can search Arxiv for papers related to a given topic, including abstracts",
    system_message="You are a helpful AI assistant. Solve tasks using your tools. Specifically, you can take into consideration the user's request and craft a search query that is most likely to return relevant academi papers.",
)

**2. Reader Agent (Document Processor)**

    Purpose: Reads, parses, and extracts key information from research papers.
    Responsibilities:
    1.   Extracting abstracts, keywords, and main sections (introduction, methodology, results, discussion).
    2.   Identifying key arguments, methodologies, and findings.
    3.   Converting scanned PDFs to machine-readable text (OCR, if needed).

In [ ]:
report_agent = AssistantAgent(
    name="Report_Agent",
    model_client = OpenAIChatCompletionClient(
      model="llama-3.3-70b-versatile",
      base_url="https://api.groq.com/openai/v1",
      api_key=os.environ["GROQ_API_KEY"],
      model_info={
        "vision": False,
        "function_calling": True,
        "json_output": False,
        "family": "unknown",
      },
    ),
    description="Generate a report based on a given topic",
    system_message="You are a helpful assistant. Your task is to get the papers info mostly only titles from another agent you're gonna select only recent one and give it back to the user, user might ask questions you still need to answer their questions.",
)

**5. Human Feedback Agent (Interactive Editor)**

    Purpose: Allows human users to provide feedback and modify the generated text.
    Responsibilities:
        1.   Displaying the draft for user review.
        2.   Accepting edits, comments, and suggestions.
        3.   Incorporating changes into the final version.

In [ ]:
user_proxy = UserProxyAgent("user_proxy", input_func=input)

Finally, we will create a team of agents and configure them to perform the tasks.

In [ ]:
text_mention_termination = TextMentionTermination("TERMINATE")
max_messages_termination = MaxMessageTermination(max_messages=25)
termination = text_mention_termination | max_messages_termination


team = RoundRobinGroupChat(
    participants=[arxiv_search_agent, report_agent, user_proxy], termination_condition=termination
)

NameError: name 'arxiv_search_agent' is not defined

In [ ]:
await Console(
    team.run_stream(
        task="Give me name of some research papers reagrding no code tools",
    )
)

---------- user ----------
Give me name of some research papers reagrding no code tools
************ calling---------- Arxiv_Search_Agent ----------

[FunctionCall(id='call_asaz', arguments='{"query": "no code tools research papers", "max_results": 5}', name='arxiv_search')]
---------- Arxiv_Search_Agent ----------
[FunctionExecutionResult(content='[{\'title\': "Ease on Down the Code: Complex Collaborative Qualitative Coding Simplified with \'Code Wizard\'"}, {\'title\': \'Bridging the Gap: A Survey and Classification of Research-Informed Ethical Hacking Tools\'}, {\'title\': \'A Tool for In-depth Analysis of Code Execution Reasoning of Large Language Models\'}, {\'title\': \'Deep Learning-Based Video Coding: A Review and A Case Study\'}, {\'title\': \'Code Swarm: A Code Generation Tool Based on the Automatic Derivation of Transformation Rule Set\'}]', call_id='call_asaz', is_error=False)]
---------- Arxiv_Search_Agent ----------
[{'title': "Ease on Down the Code: Complex Collaborative

TaskResult(messages=[TextMessage(source='user', models_usage=None, content='Give me name of some research papers reagrding no code tools', type='TextMessage'), ToolCallRequestEvent(source='Arxiv_Search_Agent', models_usage=RequestUsage(prompt_tokens=1689, completion_tokens=26), content=[FunctionCall(id='call_asaz', arguments='{"query": "no code tools research papers", "max_results": 5}', name='arxiv_search')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(source='Arxiv_Search_Agent', models_usage=None, content=[FunctionExecutionResult(content='[{\'title\': "Ease on Down the Code: Complex Collaborative Qualitative Coding Simplified with \'Code Wizard\'"}, {\'title\': \'Bridging the Gap: A Survey and Classification of Research-Informed Ethical Hacking Tools\'}, {\'title\': \'A Tool for In-depth Analysis of Code Execution Reasoning of Large Language Models\'}, {\'title\': \'Deep Learning-Based Video Coding: A Review and A Case Study\'}, {\'title\': \'Code Swarm: A Code Generation T

In [34]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.base import Handoff
from autogen_agentchat.conditions import HandoffTermination, TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient

text_termination = TextMentionTermination("TERMINATE")
max_messages_termination = MaxMessageTermination(max_messages=7)
combined_termination = max_messages_termination | text_termination

custom_model_client = OpenAIChatCompletionClient(
    model="mixtral-8x7b-32768",
    base_url="https://api.groq.com/openai/v1",
    api_key=os.environ["GROQ_API_KEY"],
    model_info={
        "vision": False,
        "function_calling": True,
        "json_output": False,
        "family": "unknown",
    },
)

from typing import List

def search_arxiv_and_chunk_pdfs(query: str) -> int:
    """
    Searches arXiv for papers related to the given query, downloads the PDFs,
    and returns the number of chunks generated from the PDFs.

    Args:
        query (str): The research topic to search for on arXiv.

    Returns:
        int: The number of chunks generated from the PDFs of the found papers.
    """
    # Placeholder for the actual implementation
    chunks = []  # Assume this is populated with chunks from the PDFs
    return len(chunks)

arxiv_search_tool = FunctionTool(
    search_arxiv_and_chunk_pdfs,
    description="Searches arXiv for papers related to a given topic, downloads the PDFs, and returns the number of chunks generated from the PDFs."
)

research_finder_agent = AssistantAgent(
    name="ResearchFinderAgent",
    description="An agent specialized in finding and processing research papers from arXiv. This agent should be the first point of contact when initiating a new research task.",
    model_client=custom_model_client,
    tools=[arxiv_search_tool],
    system_message="""
    You are the Research Finder Agent, responsible for locating and processing research papers from arXiv based on a given topic.
    When provided with a research topic, you will use the `search_arxiv_and_chunk_pdfs` tool to search arXiv, download the relevant PDFs, and generate chunks from them.
    Your primary task is to pass the research topic to the tool and return the number of chunks generated.
    Ensure that the query is clear and specific to yield the most relevant results.
    """
)

summarization: List[str] = []

def get_chunks_by_index(start_index: int) -> List[str]:
    return chunks[start_index : start_index + 10]

def save_summarization(summary: str) -> None:
    global summarization
    summarization.append(summary)

get_chunks_tool = FunctionTool(
    get_chunks_by_index,
    description="Retrieves 10 chunks of research paper content starting from a given index. Use this tool to fetch chunks in batches."
)

save_summarization_tool = FunctionTool(
    save_summarization,
    description="Saves the generated summarization to a global variable. Use this tool to store the summarized content."
)

summarization_agent = AssistantAgent(
    name="SummarizationAgent",
    description="An agent responsible for summarizing research paper chunks in batches of 10. It retrieves chunks using a tool and saves the summaries to a global variable.",
    model_client=custom_model_client,
    tools=[get_chunks_tool, save_summarization_tool],
    system_message="""
    You are the Summarization Agent. Your task is to process research paper chunks in batches of 10, summarize them concisely, and save the summaries.
    Follow these steps:
    1. Use the `get_chunks_by_index` tool to retrieve 10 chunks at a time by passing the appropriate start index.
    2. Summarize the retrieved chunks briefly and precisely.
    3. Use the `save_summarization` tool to store the generated summary in the global `summarization` variable.
    4. Repeat the process until all chunks are processed.
    Ensure your summaries are clear, concise, and capture the key points of the research content.
    """
)

def get_summarization() -> List[str]:
    """
    Retrieves the global `summarization` variable containing all the summarized content.

    Returns:
        List[str]: A list of summarized content from the research paper chunks.
    """
    global summarization
    return summarization

# Define the tool
get_summarization_tool = FunctionTool(
    get_summarization,
    description="Retrieves the summarized content from the global `summarization` variable. Use this tool to access the summaries for writing a literature review."
)

# Define the writer agent
writer_agent = AssistantAgent(
    name="WriterAgent",
    description="An agent responsible for writing a literature review based on the summarized content of research paper chunks. It retrieves the summaries using a tool and composes a coherent review.",
    model_client=custom_model_client,
    tools=[get_summarization_tool],
    system_message="""
    You are the Writer Agent. Your task is to write a comprehensive and coherent literature review based on the summarized content of research paper chunks.
    Follow these steps:
    1. Use the `get_summarization` tool to retrieve all the summarized content.
    2. Analyze the summaries to identify key themes, trends, and insights.
    3. Write a well-structured literature review that synthesizes the summarized content into a cohesive narrative.
    4. Ensure the literature review is clear, concise, and logically organized.
    5. Focus on highlighting the relationships between different studies, gaps in the research, and potential areas for future work.
    Your output should be a high-quality literature review suitable for academic or professional use.
    """
)

planning_agent = AssistantAgent(
    name="PlanningAgent",
    description="An agent responsible for planning and coordinating the workflow of the research team. It ensures tasks are executed in the correct sequence and handles user feedback for revisions.",
    model_client=custom_model_client,
    system_message="""
    You are the Planning Agent, responsible for orchestrating the workflow of the research team. Your team consists of the following agents:
    1. **ResearchFinderAgent**: Finds research papers related to the topic and returns the number of chunks.
    2. **SummarizationAgent**: Summarizes chunks of research papers in batches of 10.
    3. **WriterAgent**: Writes a literature review based on the summarized content.

    Follow these steps to complete the task:
    1. **Task Initialization**: Start by passing the research topic to the `ResearchFinderAgent`. It will search for papers, process the PDFs, and return the number of chunks.
    2. **Chunk Summarization**: Based on the number of chunks returned by the `ResearchFinderAgent`, calculate the chunk indices and call the `SummarizationAgent` to summarize each batch of 10 chunks.
    3. **Literature Review**: Once all chunks are summarized, call the `WriterAgent` to write a literature review using the summarized content.
    4. **User Feedback**: After the literature review is generated, present it to the user and collect their feedback.
    5. **Review Revision**: If the user provides feedback, pass it to the `WriterAgent` to rewrite the literature review based on the feedback.

    Your role is to ensure the workflow is executed sequentially and that each agent performs its task correctly. Communicate clearly with the user and handle feedback effectively.
    """
)

# Define the user proxy agent
user_proxy = UserProxyAgent(
    name="user_proxy",
    input_func=input  # Function to get user input
)

# Create the team with a RoundRobinGroupChat
lazy_agent_team = RoundRobinGroupChat(
    [planning_agent, research_finder_agent, summarization_agent, writer_agent, user_proxy],
    termination_condition=combined_termination
)

# Run the team and stream to the console
task = "Multi-agent Systems"
await Console(lazy_agent_team.run_stream(task=task))

---------- user ----------
Multi-agent Systems
---------- PlanningAgent ----------
As the Planning Agent, I will manage the workflow of the research team consisting of ResearchFinderAgent, SummarizationAgent, and WriterAgent. Here is the detailed plan to complete the task:

1. **Task Initialization**: I will start by passing the research topic to the ResearchFinderAgent. It will find research papers, process the PDFs, and return the total number of chunks. I will then communicate this information to the user.

2. **Chunk Summarization**: Based on the number of chunks, I will calculate the chunk indices and divide them into batches of 10 chunks each. Then, I will iterate through these batches and call the SummarizationAgent to summarize each batch. Upon completion of summarizing all chunks, I will confirm with the user.

3. **Literature Review**: I will then gather all the summarized content and pass it to the WriterAgent to create a literature review. Once the literature review is gene

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='Multi-agent Systems', type='TextMessage'), TextMessage(source='PlanningAgent', models_usage=RequestUsage(prompt_tokens=344, completion_tokens=344), metadata={}, content="As the Planning Agent, I will manage the workflow of the research team consisting of ResearchFinderAgent, SummarizationAgent, and WriterAgent. Here is the detailed plan to complete the task:\n\n1. **Task Initialization**: I will start by passing the research topic to the ResearchFinderAgent. It will find research papers, process the PDFs, and return the total number of chunks. I will then communicate this information to the user.\n\n2. **Chunk Summarization**: Based on the number of chunks, I will calculate the chunk indices and divide them into batches of 10 chunks each. Then, I will iterate through these batches and call the SummarizationAgent to summarize each batch. Upon completion of summarizing all chunks, I will confirm with

In [ ]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.base import Handoff
from autogen_agentchat.conditions import HandoffTermination, TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient

custom_model_client = OpenAIChatCompletionClient(
    model="llama-3.3-70b-versatile",
    base_url="https://api.groq.com/openai/v1",
    api_key=os.environ["GROQ_API_KEY"],
    model_info={
        "vision": False,
        "function_calling": True,
        "json_output": False,
        "family": "unknown",
    },
)

# Create a lazy assistant agent that always hands off to the user.
lazy_agent = AssistantAgent(
    "assistant",
    model_client=custom_model_client,
    system_message="help the user with his questions",
)

# Define a termination condition that checks for handoff message targetting helper and text "TERMINATE".
# handoff_termination = HandoffTermination(target="user")
text_termination = TextMentionTermination("TERMINATE")
max_messages_termination = MaxMessageTermination(max_messages=7)
combined_termination = max_messages_termination | text_termination

# Create a single-agent team.
user_proxy = UserProxyAgent("user_proxy", input_func=input)
lazy_agent_team = RoundRobinGroupChat([lazy_agent, user_proxy], termination_condition=combined_termination)

# Run the team and stream to the console.
task = "What is the weather in New York?"
await Console(lazy_agent_team.run_stream(task=task))

---------- user ----------
What is the weather in New York?
---------- assistant ----------
I'm a large language model, I don't have have access to real-time information, so I can't provide you with the current weather in New York. However, I can suggest some ways for you to find out the current weather in New York:

1. **Check online weather websites**: You can visit websites like weather.com, accuweather.com, or wunderground.com to get the current weather conditions in New York.
2. **Use a weather app**: You can download a weather app on your smartphone, such as Dark Sky or Weather Underground, to get real-time weather updates for New York.
3. **Check social media**: You can check the official social media accounts of the National Weather Service (NWS) or the New York City government to get updates on the weather.
4. **Turn on the TV**: You can watch the local news or weather channel on TV to get the current weather conditions in New York.

If you're looking for general information a

TaskResult(messages=[TextMessage(source='user', models_usage=None, content='What is the weather in New York?', type='TextMessage'), TextMessage(source='assistant', models_usage=RequestUsage(prompt_tokens=49, completion_tokens=280), content="I'm a large language model, I don't have have access to real-time information, so I can't provide you with the current weather in New York. However, I can suggest some ways for you to find out the current weather in New York:\n\n1. **Check online weather websites**: You can visit websites like weather.com, accuweather.com, or wunderground.com to get the current weather conditions in New York.\n2. **Use a weather app**: You can download a weather app on your smartphone, such as Dark Sky or Weather Underground, to get real-time weather updates for New York.\n3. **Check social media**: You can check the official social media accounts of the National Weather Service (NWS) or the New York City government to get updates on the weather.\n4. **Turn on the T

In [ ]:
text_mention_termination = TextMentionTermination("TERMINATE")
max_messages_termination = MaxMessageTermination(max_messages=4)
termination = text_mention_termination | max_messages_termination

# Note: This example uses mock tools instead of real APIs for demonstration purposes
# def search_web_tool(query: str) -> str:
#     if "2006-2007" in query:
#         return """Here are the total points scored by Miami Heat players in the 2006-2007 season:
#         Udonis Haslem: 844 points
#         Dwayne Wade: 1397 points
#         James Posey: 550 points
#         ...
#         """
#     elif "2007-2008" in query:
#         return "The number of total rebounds for Dwayne Wade in the Miami Heat season 2007-2008 is 214."
#     elif "2008-2009" in query:
#         return "The number of total rebounds for Dwayne Wade in the Miami Heat season 2008-2009 is 398."
#     return "No data found."


# def percentage_change_tool(start: float, end: float) -> float:
#     return ((end - start) / start) * 100




model_client = OpenAIChatCompletionClient(
    # model="llama-3.3-70b-versatile",
    # model="llama-3.1-8b-instant",
    model="llama3-70b-8192",
    base_url="https://api.groq.com/openai/v1",
    api_key=os.environ["GROQ_API_KEY"],
    model_info={
        "vision": False,
        "function_calling": True,
        "json_output": False,
        "family": "unknown",
    },
)

planning_agent = AssistantAgent(
    "PlanningAgent",
    description="An agent for planning tasks, this agent should be the first to engage when given a new task.",
    model_client=model_client,
    system_message="""
    You are a planning agent.
    Your job is to make sure agents work sequentially one after each other we have only two
    Your team members are:
        ResearchAgent: Paper Retriever
        ReaderAgent: Document Processor

    You only plan and delegate tasks - you do not execute them yourself.

    After all tasks are complete, summarize the findings.
    """,
)

# web_search_agent = AssistantAgent(
#     "WebSearchAgent",
#     description="An agent for searching information on the web.",
#     tools=[search_web_tool],
#     model_client=model_client,
#     system_message="""
#     You are a web search agent.
#     Your only tool is search_tool - use it to find information.
#     You make only one search call at a time.
#     Once you have the results, you never do calculations based on them.
#     """,
# )

# data_analyst_agent = AssistantAgent(
#     "DataAnalystAgent",
#     description="An agent for performing calculations.",
#     model_client=model_client,
#     tools=[percentage_change_tool],
#     system_message="""
#     You are a data analyst.
#     Given the tasks you have been assigned, you should analyze the data and provide results using the tools provided.
#     If you have not seen the data, ask for it.
#     """,
# )




# 1

def research_arxiv_tool(query: str) -> str:
    import arxiv

    # Construct the default API client.
    client = arxiv.Client()

    # Search for the 2 most recent articles matching the query.
    search = arxiv.Search(
        query=query,
        max_results=1,
    )

    results_list = []

    for result in client.results(search):
        results_list.append(f"Title: {result.title}\nSummary: {result.summary}\n")

    return "\n".join(results_list) if results_list else "No results found."

research_agent = AssistantAgent(
    "ResearchAgent",
    description="You are an AI-powered assistant that extracts concise and effective search queries for academic research. Given a user input, it refines the request into a well-structured query that optimizes the ArXiv search tool for relevant results.",
    tools=[research_arxiv_tool],
    model_client=model_client,
    system_message="""
    Your only task is to extract a query not longer that 3 words.
    You do not summarize, analyze, or modify the research papers.
    Your job is only to construct the best possible query for the research_arxiv_tool.
    ONLY EXTRACT THREE WORDS FOR OUR TOOL, ONLY THREE(WORDS) TO FEED OUR FUNCTION, CHOOSE MOST RELEVANT KEYWORDS AND RUN THE TOOL WITH IT
    """,
)

reader_agent = AssistantAgent(
    "ReaderAgent",
    description="An AI-powered research assistant designed to analyze academic papers and extract key information. It helps users quickly understand the core content of a research paper without reading the full document.",
    model_client=model_client,
    system_message="""
    You are an intelligent document processing agent specialized in academic research.
    Your task is to **read, parse, and extract key information** from research papers.
    You DO NOT summarize arbitrarily—your goal is structured extraction of information.

    For every research paper you process, extract and return:
    - **Title**
    - **Abstract**
    - **Keywords**

    Maintain accuracy and structure. Do not add opinions or assumptions.
    If a section is missing or unclear, mention it without making up content.
    Your job is purely **document analysis and structured extraction**, not interpretation.
    """,
)

summarization_agent = AssistantAgent(
    "SummarizationAgent",
    description="An AI-powered research assistant that generates structured summaries of research papers. It extracts key insights, highlights major contributions, and identifies gaps in the literature while organizing findings into thematic categories.",
    model_client=model_client,
    system_message="""
    You are an advanced research summarization agent.
    Your job is to generate **structured summaries** of research papers and extract key insights.
    You DO NOT simply copy abstracts—you summarize in an organized and meaningful way.

    For every paper, produce a structured summary including:
    - **Title**
    - **Core Summary**: (A concise, 3-5 sentence explanation of the paper)
    - **Major Contributions**: (What new insights, methods, or findings does this paper offer?)
    - **Identified Gaps**: (What areas remain unexplored or need further research?)
    - **Categorized Findings**: (If possible, organize the insights into themes or subtopics)

    Your goal is to make research **accessible and structured** while preserving accuracy.
    Avoid unnecessary details or overly technical jargon unless necessary.
    """,
)


selector_prompt = """Select an agent to perform task.

{roles}

Current conversation context:
{history}

Read the above conversation, then select an agent from {participants} to perform the next task.
Make sure the planner agent has assigned tasks before other agents start working.
Only select one agent.
"""


team = SelectorGroupChat(
    [planning_agent, research_agent, reader_agent],
    model_client=model_client,
    termination_condition=termination,
    selector_prompt=selector_prompt,
    allow_repeated_speaker=True,  # Allow an agent to speak multiple turns in a row.
)

task = "I am researching the latest advancements in Multi-Agent Large Language Models (LLMs). Specifically, I want to understand how multiple LLMs can collaborate to solve complex tasks, including coordination strategies, communication methods, and distributed problem-solving. Please find relevant research papers, extract key insights, and summarize major contributions and gaps in this field."

# Use asyncio.run(...) if you are running this in a script.
await Console(team.run_stream(task=task))

---------- user ----------
I am researching the latest advancements in Multi-Agent Large Language Models (LLMs). Specifically, I want to understand how multiple LLMs can collaborate to solve complex tasks, including coordination strategies, communication methods, and distributed problem-solving. Please find relevant research papers, extract key insights, and summarize major contributions and gaps in this field.
---------- PlanningAgent ----------
A fascinating topic! I'll delegate tasks to my team members to help you with this research. Here's the plan:

**Task 1: Research Paper Retrieval**

I'll assign the ResearchAgent (Paper Retriever) to find relevant research papers related to Multi-Agent Large Language Models (LLMs) and their collaboration strategies. The agent will search through academic databases, conference proceedings, and online repositories to retrieve a list of papers that match the specified topic.

**Task 2: Document Processing and Insights Extraction**

Once the papers

TaskResult(messages=[TextMessage(source='user', models_usage=None, content='I am researching the latest advancements in Multi-Agent Large Language Models (LLMs). Specifically, I want to understand how multiple LLMs can collaborate to solve complex tasks, including coordination strategies, communication methods, and distributed problem-solving. Please find relevant research papers, extract key insights, and summarize major contributions and gaps in this field.', type='TextMessage'), TextMessage(source='PlanningAgent', models_usage=RequestUsage(prompt_tokens=156, completion_tokens=294), content="A fascinating topic! I'll delegate tasks to my team members to help you with this research. Here's the plan:\n\n**Task 1: Research Paper Retrieval**\n\nI'll assign the ResearchAgent (Paper Retriever) to find relevant research papers related to Multi-Agent Large Language Models (LLMs) and their collaboration strategies. The agent will search through academic databases, conference proceedings, and 

In [ ]:
await team.reset()

In [ ]:
literature_review = """
The integration of Large Language Models (LLMs) into multi-agent systems has recently garnered significant attention, aiming to enhance collaborative problem-solving and dynamic interaction capabilities. This literature review examines key advancements in this domain, focusing on system architectures, communication strategies, and application areas.

**System Architectures and Frameworks**

A notable contribution is the "Chain-of-Agents" (CoA) framework, which addresses the challenge of processing extensive contexts by enabling multiple LLMs to collaborate on long-context tasks. This training-free, interpretable approach has demonstrated superior performance over traditional models like Retrieval-Augmented Generation (RAG) and long-context LLMs, particularly in handling inputs exceeding 100,000 tokens. The CoA framework's design emphasizes cost-effectiveness and task-agnostic applicability, making it a versatile solution for complex language processing tasks. citeturn0search5

Another significant development is "LongAgent," a method that scales LLMs to a 128K context through multi-agent collaboration. In this setup, a leader agent interprets user intent and directs member agents to extract pertinent information from documents. To mitigate issues like hallucinations among member agents, LongAgent incorporates an inter-member communication mechanism that resolves conflicts through information sharing. Empirical results indicate that LongAgent outperforms models like GPT-4 in tasks such as long-text retrieval and multi-hop question answering, highlighting its efficacy in managing extensive textual data. citeturn0academia11

**Evaluation and Benchmarking**

The "LLMArena" framework offers a novel approach to assessing LLMs within dynamic multi-agent environments. It encompasses seven distinct gaming environments, utilizing Trueskill scoring to evaluate abilities such as spatial reasoning, strategic planning, numerical reasoning, risk assessment, communication, opponent modeling, and team collaboration. Studies using LLMArena have revealed that current LLMs face challenges in opponent modeling and team collaboration, underscoring areas for future enhancement in developing autonomous agents. citeturn0academia12

**Communication and Coordination Strategies**

The "MAgIC" framework delves into the cognitive and collaborative dimensions of LLM-powered multi-agent systems. By employing games like Chameleon and Undercover, alongside game theory scenarios such as Cost Sharing and the Multi-player Prisoner's Dilemma, MAgIC evaluates agents' judgment, reasoning, deception, self-awareness, cooperation, coordination, and rationality. The integration of Probabilistic Graphical Modeling (PGM) enhances agents' capabilities in navigating complex social interactions, providing a comprehensive assessment of their cognitive and collaborative proficiencies. citeturn0academia10

**Applications in Software Engineering**

The application of LLM-based multi-agent systems in software engineering has been explored to address intricate challenges in the field. These systems leverage the collaborative potential of multiple LLMs to enhance software development processes, including code synthesis, debugging, and project management. By distributing tasks among specialized agents, these systems aim to improve efficiency and accuracy in software engineering tasks, though further empirical studies are necessary to validate their effectiveness in real-world scenarios. citeturn0search6

**Challenges and Future Directions**

Despite the promising advancements, several challenges persist in the development of LLM-based multi-agent systems. Ensuring logical consistency, managing hallucinations, and enhancing inter-agent communication remain critical areas for improvement. Future research is poised to focus on refining these aspects, developing robust evaluation frameworks, and exploring diverse application domains to fully harness the potential of collaborative LLMs in complex problem-solving contexts.

In summary, the integration of LLMs into multi-agent systems represents a burgeoning field with the potential to revolutionize collaborative artificial intelligence. Ongoing research continues to address existing challenges, paving the way for more sophisticated and effective multi-agent collaborations in various domains.
"""

In [ ]:
text_mention_termination = TextMentionTermination("TERMINATE")
max_messages_termination = MaxMessageTermination(max_messages=5)
termination = text_mention_termination | max_messages_termination

user_proxy = UserProxyAgent("user_proxy", input_func=input)

model_client = OpenAIChatCompletionClient(
    model="llama-3.3-70b-versatile",
    # model="llama-3.1-8b-instant",
    # model="llama3-70b-8192",
    base_url="https://api.groq.com/openai/v1",
    api_key=os.environ["GROQ_API_KEY"],
    model_info={
        "vision": False,
        "function_calling": True,
        "json_output": False,
        "family": "unknown",
    },
)

def get_review() -> str:
    return literature_review

get_review_agent = AssistantAgent(
    "GetReviewAgent",
    description="An agent for getting the orginal review.",
    tools=[get_review],
    model_client=model_client,
    system_message="""
    You are a planning agent.
    Your job is to get the original review from the tool you have and just return it without any changes.
    """,
)

planning_agent = AssistantAgent(
    "PlanningAgent",
    description="An agent for planning tasks, this agent should be the first to engage when given a new task.",
    model_client=model_client,
    system_message="""
    You are a planning agent.
    Your job is to make sure agents work sequentially one after each other we have only two
    Your team members are:
        GetReviewAgent: Give us first original review
        UserAgent: User Feedback On The Review
        RefinementAgent: Refines the output based on user feedback

    You only plan and delegate tasks - you do not execute them yourself.

    After all tasks are complete, summarize the findings.
    """,
)

refinement_agent = AssistantAgent(
    "RefinementAgent",
    description="An AI-powered quality assurance assistant that improves the clarity, coherence, and consistency of literature reviews. It refines the output based on user feedback, ensuring logical consistency, grammatical correctness, and readability.",
    model_client=model_client,
    system_message="""
    You are a Quality Assurance and Refinement Agent specializing in academic literature review.
    Your primary role is to refine summaries based on user feedback, ensuring they are **clear, coherent, and logically structured**.

    For each refinement request:
    - **Improve clarity** by restructuring sentences where necessary.
    - **Ensure logical flow** between sections and arguments.
    - **Fix grammar, spelling, and formatting issues** without changing the original meaning.
    - **Maintain accuracy** while making the content easier to understand.
    - **Do not introduce new information**—your job is refinement, not expansion.

    If user feedback is vague, make **general readability improvements** while preserving meaning.
    """,
)


selector_prompt = """Select an agent to perform task.

{roles}

Current conversation context:
{history}

Read the above conversation, then select an agent from {participants} to perform the next task.
Make sure the planner agent has assigned tasks before other agents start working.
Only select one agent.
"""


team = SelectorGroupChat(
    [planning_agent, get_review_agent, user_proxy, refinement_agent],
    model_client=model_client,
    termination_condition=termination,
    selector_prompt=selector_prompt,
    allow_repeated_speaker=True,  # Allow an agent to speak multiple turns in a row.
)

task = ""

# Use asyncio.run(...) if you are running this in a script.
await Console(team.run_stream(task=task))

---------- user ----------

---------- PlanningAgent ----------
To begin the process, I will delegate tasks to the agents in a sequential manner. Here's the plan:

1. **GetReviewAgent**: I assign the task to GetReviewAgent to provide the first original review. This review will serve as the foundation for further refinement.

Once GetReviewAgent completes the task, I will delegate the next task.

2. **UserAgent**: After receiving the original review from GetReviewAgent, I will assign the task to UserAgent to provide feedback on the review. This feedback is crucial for identifying areas of improvement.

After UserAgent completes the task, I will delegate the final task.

3. **RefinementAgent**: With the feedback from UserAgent, I will assign the task to RefinementAgent to refine the output based on the user's feedback. This step aims to enhance the quality of the review.

Once all tasks are complete, I will summarize the findings.

**Waiting for task completion...**

Please let me know w

TaskResult(messages=[TextMessage(source='user', models_usage=None, content='', type='TextMessage'), TextMessage(source='PlanningAgent', models_usage=RequestUsage(prompt_tokens=127, completion_tokens=225), content="To begin the process, I will delegate tasks to the agents in a sequential manner. Here's the plan:\n\n1. **GetReviewAgent**: I assign the task to GetReviewAgent to provide the first original review. This review will serve as the foundation for further refinement.\n\nOnce GetReviewAgent completes the task, I will delegate the next task.\n\n2. **UserAgent**: After receiving the original review from GetReviewAgent, I will assign the task to UserAgent to provide feedback on the review. This feedback is crucial for identifying areas of improvement.\n\nAfter UserAgent completes the task, I will delegate the final task.\n\n3. **RefinementAgent**: With the feedback from UserAgent, I will assign the task to RefinementAgent to refine the output based on the user's feedback. This step a

In [ ]:
!pip install numpy -q
!pip install nltk -q
!pip install rouge -q
!pip install bert_score -q
!pip install textstat -q
!pip install spacy -q
!pip install torch -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 89.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 80.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 98.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.3/105.3 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9

In [ ]:
import numpy as np
import nltk
import rouge
import bert_score
import textstat
import spacy
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.translate.bleu_score import sentence_bleu

# Download required resources
nltk.download('punkt')
nlp = spacy.load("en_core_web_sm")

# Load GPT-2 for perplexity calculation
gpt2_tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
gpt2_model = GPT2LMHeadModel.from_pretrained("gpt2")

def compute_rouge_scores(reference, generated):
    evaluator = rouge.Rouge()
    scores = evaluator.get_scores(generated, reference)
    return scores

def compute_bleu(reference, generated):
    reference_tokens = [nltk.word_tokenize(reference)]
    generated_tokens = nltk.word_tokenize(generated)
    return sentence_bleu(reference_tokens, generated_tokens)

def compute_bert_score(reference, generated):
    P, R, F1 = bert_score.score([generated], [reference], lang="en")
    return {'precision': P.mean().item(), 'recall': R.mean().item(), 'f1': F1.mean().item()}

def compute_readability(text):
    return textstat.flesch_reading_ease(text)

def compute_perplexity(text):
    tokens = gpt2_tokenizer.encode(text, return_tensors="pt")
    with torch.no_grad():
        loss = gpt2_model(tokens, labels=tokens).loss
    return torch.exp(loss).item()

def compute_cosine_similarity(reference, generated):
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform([reference, generated])
    similarity = cosine_similarity(tfidf_matrix[0], tfidf_matrix[1])[0][0]
    return similarity

def extract_cited_entities(text):
    doc = nlp(text)
    citations = [ent.text for ent in doc.ents if ent.label_ in ['PERSON', 'ORG', 'WORK_OF_ART']]
    return set(citations)

def evaluate_reviews(reference_text, generated_text):
    scores = {}

    scores['rouge'] = compute_rouge_scores(reference_text, generated_text)
    scores['bleu'] = compute_bleu(reference_text, generated_text)
    scores['bert_score'] = compute_bert_score(reference_text, generated_text)
    scores['cosine_similarity'] = compute_cosine_similarity(reference_text, generated_text)
    scores['readability'] = compute_readability(generated_text)
    scores['perplexity'] = compute_perplexity(generated_text)

    reference_citations = extract_cited_entities(reference_text)
    generated_citations = extract_cited_entities(generated_text)
    scores['citation_overlap'] = len(reference_citations & generated_citations) / max(1, len(reference_citations))

    return scores

if __name__ == "__main__":
    with open("human_review.txt", "r", encoding="utf-8") as f:
        reference_text = f.read()
    with open("llm_review.txt", "r", encoding="utf-8") as f:
        generated_text = f.read()

    results = evaluate_reviews(reference_text, generated_text)
    print(results)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

FileNotFoundError: [Errno 2] No such file or directory: 'human_review.txt'